In [11]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/master_degree/paper')
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 논문계획서_2nd.gdoc			  data
 archive.zip				 '기존 분석결과.gdoc'
 booking.com_hotel_review_korea.csv.zip  '논문 지도교수 배정신청서.gdoc'
 code_R					 '레퍼런스 링크.gdoc'
 colab


In [12]:
import pandas as pd
import pickle
import json
from io import StringIO
import sys

In [19]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords') 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [155]:
# stopwords 점검
original_stops = stopwords.words('english')
print(len(original_stops))
my_stops = ['zoom', 'fbm'] + original_stops
print(my_stops)

179
['zoom', 'fbm', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'sam

In [21]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    # keywords= [lemma for lemma in lemmas if lemma not in my_stops] # apply customized stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english'] 
    return keywords

def build_freqs(text):
    freqs = {}
    for word in preprocess_text(text):
        if word in freqs:
            freqs[word] += 1
        else:
            freqs[word] = 1
    return freqs

In [99]:
df = pd.read_pickle('data/processed/rv_en_1.pkl')

In [102]:
print(df.info())
df.reset_index(inplace=True, drop=True)
df.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250624 entries, 0 to 1250623
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   app_nm   1250624 non-null  object 
 1   review   1250624 non-null  object 
 2   rating   1250624 non-null  float64
 3   dt       1250624 non-null  object 
 4   app_ver  1250624 non-null  object 
 5   lang     1250624 non-null  object 
dtypes: float64(1), object(5)
memory usage: 57.2+ MB
None


,app_nm,review,rating,dt,app_ver,lang
1250614,fbm,Cool way to communicate,4.0,2020-12-23,294.0.0.20.129,en
1250615,fbm,Good,3.0,2020-12-26,288.0.0.15.118,en
1250616,fbm,It's the worst ever it hangs and as i reboot m...,5.0,2020-12-24,257.1.0.21.120,en
1250617,fbm,No se puede dibujar sobre los las esquinas el ...,5.0,2020-12-27,294.0.0.20.129,en
1250618,fbm,You can't see pictures and videos you have sen...,3.0,2020-12-24,294.0.0.24.129,en
1250619,fbm,problem in archive story and how many other se...,1.0,2020-12-25,290.0.0.16.119,en
1250620,fbm,why isn't there a way to change an individual'...,1.0,2020-12-28,294.0.0.24.129,en
1250621,fbm,The latest update ruined this app... The chat ...,1.0,2020-12-28,294.0.0.24.129,en
1250622,fbm,Since Android 11 the app is baaaad. That or it...,2.0,2020-12-26,294.0.0.24.129,en
1250623,fbm,Censor this Facebook!!!🖕🖕🖕🖕🖕,1.0,2020-12-23,283.0.0.16.120,en


# Dataset describe

In [103]:
print(df['app_nm'].unique())
print('Number of apps: ',len(df['app_nm'].unique()))

['zoom' 'discord' 'fbm']
Number of apps:  3


In [104]:
print('rows and columns', df.shape, '\n')
print('reviews of each app', df.groupby(['app_nm']).size())
print('reviews of each app', df.groupby(['app_nm']).count())

rows and columns (1250624, 6) 

reviews of each app app_nm
discord    118292
fbm        795610
zoom       336722
dtype: int64
reviews of each app          review  rating      dt  app_ver    lang
app_nm                                          
discord  118292  118292  118292   118292  118292
fbm      795610  795610  795610   795610  795610
zoom     336722  336722  336722   336722  336722


# WHAT TO DO?
### preprocessing

reference 1: https://towardsdatascience.com/introduction-to-nlp-part-1-preprocessing-text-in-python-8f007d44ca96

reference 2: https://bkshin.tistory.com/entry/NLP-4-%EC%96%B4%EA%B0%84-%EC%B6%94%EC%B6%9CStemming%EA%B3%BC-%ED%91%9C%EC%A0%9C%EC%96%B4-%EC%B6%94%EC%B6%9CLemmatization

* tokenize
* lemmatize
* build frequencies

In [105]:
# 리뷰 텍스트만 list 로 추출
rv = list(df['review'])
print(len(rv))

1250624


In [107]:
# 키워드별 count (dict 형태)
rv_dict = dict()
for i in range(len(rv)):
  if i%10000==0:
    print(i, end=" - ")
  if i%100000==0:
    print('\n')
  rv_dict[i] = build_freqs(rv[i])

0 - 

10000 - 20000 - 30000 - 40000 - 50000 - 60000 - 70000 - 80000 - 90000 - 100000 - 

110000 - 120000 - 130000 - 140000 - 150000 - 160000 - 170000 - 180000 - 190000 - 200000 - 

210000 - 220000 - 230000 - 240000 - 250000 - 260000 - 270000 - 280000 - 290000 - 300000 - 

310000 - 320000 - 330000 - 340000 - 350000 - 360000 - 370000 - 380000 - 390000 - 400000 - 

410000 - 420000 - 430000 - 440000 - 450000 - 460000 - 470000 - 480000 - 490000 - 500000 - 

510000 - 520000 - 530000 - 540000 - 550000 - 560000 - 570000 - 580000 - 590000 - 600000 - 

610000 - 620000 - 630000 - 640000 - 650000 - 660000 - 670000 - 680000 - 690000 - 700000 - 

710000 - 720000 - 730000 - 740000 - 750000 - 760000 - 770000 - 780000 - 790000 - 800000 - 

810000 - 820000 - 830000 - 840000 - 850000 - 860000 - 870000 - 880000 - 890000 - 900000 - 

910000 - 920000 - 930000 - 940000 - 950000 - 960000 - 970000 - 980000 - 990000 - 1000000 - 

1010000 - 1020000 - 1030000 - 1040000 - 1050000 - 1060000 - 1070000 - 1080000 - 10

In [109]:
# word 추출 (list)
keywords_l = []
for i in range(len(rv)):
  a = list(rv_dict[i].keys())
  keywords_l.append(a)

In [110]:
# 역토큰화 (list)
detoken_1 = []
for i in range(len(rv)):
  a = ' '.join(keywords_l[i])
  detoken_1.append(a)

In [138]:
# key value (list)
items_l = []
for i in range(len(rv)):
  a = rv_dict[i]
  items_l.append(a)

In [139]:
df2 = pd.DataFrame.from_dict({'keywords':keywords_l, 
                              'detoken':detoken_1,
                              'items': items_l})

In [143]:
df_m = pd.merge(df, df2, left_index=True, right_index=True)

In [144]:
df_m

,app_nm,review,rating,dt,app_ver,lang,keywords,detoken,items
0,zoom,ভাল,2.0,2020-12-28,5.2.45092.0831,en,"[ভ, ল]",ভ ল,"{'ভ': 1, 'ল': 1}"
1,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown,en,"[one, education, thing, cant, unlike, never]",one education thing cant unlike never,"{'one': 1, 'education': 1, 'thing': 1, 'cant':..."
2,zoom,Good,5.0,2020-12-26,5.4.6.812,en,[good],good,{'good': 1}
3,zoom,Worst appp,2.0,2020-12-28,Unknown,en,"[worst, appp]",worst appp,"{'worst': 1, 'appp': 1}"
4,zoom,Good,4.0,2020-12-28,5.4.7.946,en,[good],good,{'good': 1}
...,...,...,...,...,...,...,...,...,...
1250619,fbm,problem in archive story and how many other se...,1.0,2020-12-25,290.0.0.16.119,en,"[problem, archive, story, many, see]",problem archive story many see,"{'problem': 1, 'archive': 1, 'story': 2, 'many..."
1250620,fbm,why isn't there a way to change an individual'...,1.0,2020-12-28,294.0.0.24.129,en,"[way, change, individual, notification, sound,...",way change individual notification sound whole...,"{'way': 1, 'change': 2, 'individual': 1, 'noti..."
1250621,fbm,The latest update ruined this app... The chat ...,1.0,2020-12-28,294.0.0.24.129,en,"[latest, update, ruin, app, chat, bubble, suck...",latest update ruin app chat bubble suck notifi...,"{'latest': 1, 'update': 1, 'ruin': 1, 'app': 1..."
1250622,fbm,Since Android 11 the app is baaaad. That or it...,2.0,2020-12-26,294.0.0.24.129,en,"[since, android, 11, app, baaaad, update, noti...",since android 11 app baaaad update notice chat...,"{'since': 1, 'android': 1, '11': 1, 'app': 1, ..."


In [145]:
df_m.to_pickle('data/processed/rv_en_2.pkl')

# TF-IDF, Topic modeling

In [163]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [146]:
df = pd.read_pickle('data/processed/rv_en_2.pkl')
df.head(10)

,app_nm,review,rating,dt,app_ver,lang,keywords,detoken,items
0,zoom,ভাল,2.0,2020-12-28,5.2.45092.0831,en,"[ভ, ল]",ভ ল,"{'ভ': 1, 'ল': 1}"
1,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown,en,"[one, education, thing, cant, unlike, never]",one education thing cant unlike never,"{'one': 1, 'education': 1, 'thing': 1, 'cant':..."
2,zoom,Good,5.0,2020-12-26,5.4.6.812,en,[good],good,{'good': 1}
3,zoom,Worst appp,2.0,2020-12-28,Unknown,en,"[worst, appp]",worst appp,"{'worst': 1, 'appp': 1}"
4,zoom,Good,4.0,2020-12-28,5.4.7.946,en,[good],good,{'good': 1}
5,zoom,Pro,5.0,2020-12-26,5.2.42588.0803,en,[pro],pro,{'pro': 1}
6,zoom,It's very very useful for school students,5.0,2020-12-26,5.4.7.946,en,"[useful, school, students]",useful school students,"{'useful': 1, 'school': 1, 'students': 1}"
7,zoom,Max 4 people on one screen including myself. T...,1.0,2020-12-23,Unknown,en,"[max, 4, people, one, screen, include, usable]",max 4 people one screen include usable,"{'max': 1, '4': 1, 'people': 1, 'one': 1, 'scr..."
8,zoom,Super,5.0,2020-12-23,5.4.4.615,en,[super],super,{'super': 1}
9,zoom,Won't let me Unmute. . The level of Simplicity...,3.0,2020-12-24,Unknown,en,"[win, let, unmute, level, simplicity, complicate]",win let unmute level simplicity complicate,"{'win': 1, 'let': 1, 'unmute': 1, 'level': 1, ..."


In [161]:
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000) # 상위 1,000개의 단어를 보존 
X = vectorizer.fit_transform(df['detoken'])
print(X.shape) # TF-IDF 행렬의 크기 확인

(1250624, 1000)


In [164]:
# LDA
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

In [165]:
lda_top = lda_model.fit_transform(X)

In [166]:
print(lda_model.components_)
print(lda_model.components_.shape) 

[[1.00001433e-01 1.00001784e-01 1.00002288e-01 ... 1.00005141e-01
  1.00002522e-01 1.00000219e-01]
 [1.00007151e-01 1.00003063e-01 1.00008216e-01 ... 1.00005566e-01
  1.00005207e-01 1.00032441e-01]
 [1.00023040e-01 1.10197057e+02 1.00048783e-01 ... 1.00005371e-01
  3.93186195e+02 4.36404753e+01]
 ...
 [1.00005500e-01 1.00008203e-01 1.00007003e-01 ... 1.00041289e-01
  1.00015746e-01 1.00026498e-01]
 [1.00015545e-01 1.00012838e-01 1.00006135e-01 ... 1.00025728e-01
  1.00015321e-01 1.00013786e-01]
 [1.00013912e-01 3.36459012e+02 1.00038235e-01 ... 1.00009135e-01
  1.00007520e-01 1.00202427e-01]]
(10, 1000)


In [173]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i]) for i in topic.argsort()[:-n - 1:-1]])
        # print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: ['nice', 'ok', 'app', 'excellent', 'cool', 'work', 'download', 'win', 'poor', 'lang']
Topic 2: ['bad', 'app', 'worst', 'communication', 'fantastic', 'osm', 'error', 'massenger', 'bulok', 'life']
Topic 3: ['chat', 'really', 'fix', 'video', 'bug', 'work', 'happy', 'want', 'try', 'star']
Topic 4: ['good', 'app', 'best', 'awesome', 'amaze', 'fine', 'im', 'reason', 'stories', 'laggy']
Topic 5: ['thank', 'slow', 'suck', 'account', 'beautiful', 'hate', 'fast', 'experience', 'remove', 'free']
Topic 6: ['message', 'send', 'update', 'super', 'friends', 'time', 'fix', 'new', 'people', 'log']
Topic 7: ['love', 'messenger', 'like', 'app', 'bubble', 'better', 'na', 'receive', 'okay', 'notifications']
Topic 8: ['useful', 'nc', 'aap', 'wala', 'help', 'op', 'instal', 'terrible', 'online', 'app']
Topic 9: ['apps', 'wow', 'use', 'facebook', 'easy', 'install', 'lag', 'application', 'old', 'helpful']
Topic 10: ['great', 'update', 'new', 'app', 'effect', 'watch', 'version', 'pls', 'stupid', 'yes']


#### Gensim 으로 동일한 과정 해보기

In [175]:
from gensim import corpora

In [180]:
tokenized_doc = df['keywords']
tokenized_doc

0                                                     [ভ, ল]
1               [one, education, thing, cant, unlike, never]
2                                                     [good]
3                                              [worst, appp]
4                                                     [good]
                                 ...                        
1250619                 [problem, archive, story, many, see]
1250620    [way, change, individual, notification, sound,...
1250621    [latest, update, ruin, app, chat, bubble, suck...
1250622    [since, android, 11, app, baaaad, update, noti...
1250623                                   [censor, facebook]
Name: keywords, Length: 1250624, dtype: object

In [181]:
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]


In [183]:
print(dictionary[36])

amaze


In [184]:
len(dictionary)

178404

In [ ]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)

In [ ]:
for topic in topics:
    print(topic)
print(ldamodel.print_topics())

### LDA 시각화

In [ ]:
import pyLDAvis.gensim